<a href="https://colab.research.google.com/github/VittorioGatto98/B19-1/blob/master/ViT_con_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Eseguendo il comando sottostante si installano alcune librerie**

*   **transformers:** libreria di Hugging Face, fornisce un'interfaccia per scaricare e utilizzare modelli pre-addestrati all'avanguardia per diverse attività di Natural Language Processing (NLP), Computer Vision e Audio. Include anche l'architettura ViT che sarà utile in questo caso.unificata
*   **torch**: PyTorch è un framework di deep learning open-source ampiamente utilizzato per la ricerca e lo sviluppo di modelli di machine learning.
* **faiss-cpu**: Faiss è una libreria sviluppata da Facebook AI Research per la ricerca efficiente di similarità e l'indicizzazione di vettori ad alta dimensione. La parte -cpu indica che si sta installando la versione ottimizzata per l'utilizzo sulla CPU. È particolarmente utile per costruire database vettoriali e per effettuare ricerche di similarità veloci tra embedding.
* **Pillow**: Fornisce un'ampia gamma di funzionalità per aprire, manipolare e [testo del link](https://)salvare diversi formati di file immagine. È spesso utilizzata in progetti di computer vision per il caricamento, la visualizzazione e la pre-elaborazione di immagini prima di passarle a modelli di machine learning.

In [4]:
!pip install transformers sentence-transformers faiss-cpu Pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

**2. Definiamo un array che rappresenterà il db di immagini con relativa descrizione**

In [ ]:
# Database di immagini + descrizioni associate
db = [
    {"path": "gatto_ghibli.jpg", "desc": "Un gatto vestito da Babbo Natale"},
    {"path": "cane_ghibli.jpg", "desc": "Un cane di razza Shiba in stile Ghibli"},
    {"path": "robot_neve.jpg", "desc": "Un robot cammina in una foresta innevata"}
]

 **2. Import delle librerie utili allo script**

In [2]:
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import torch
import faiss
import numpy as np

**3. Carichiamo i modelli ViT preaddestrati**

In [3]:
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = ViTModel.from_pretrained('google/vit-base-patch16-224')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**4. Definiamo la funzione get_embedding che prenderà in ingresso un path di immagine**



In [4]:
def get_embedding(path):
    img = Image.open(path).convert("RGB") #apre l'immagine in lettura e la converte in RGB.
    inputs = processor(images=img, return_tensors="pt") #Viene usato il ViTImageProcessor per ridimensionare, normalizzare e convertire l'immagine nel formato corretto, oltre a creare un tensore "pt" da passare al modello
    with torch.no_grad():
        outputs = model(**inputs) #senza usare il gradiente, passa l'immagine pre-processata al modello ViT per ottenere l'output proveniente dal token CLS
    return outputs.last_hidden_state[:, 0].numpy().astype('float32')  # [CLS] token

**5. Costruzione del database vettoriale FAISS**

In [8]:
image_embeddings = []
descriptions = []

for item in db: #per ciascun elemento presente nel vettore db si ottiene il relativo embedding calcolato con il modello ViT
    emb = get_embedding(item["path"])
    image_embeddings.append(emb)
    descriptions.append(item["desc"])

image_embeddings = np.vstack(image_embeddings) #si uniscono i vettori ricavati prima in un'unica matrice
index = faiss.IndexFlatL2(image_embeddings.shape[1]) #si crea indice FAISS per cercare immagini simili tramite distanza L2 (euclidea).
index.add(image_embeddings)

**6. Creazione di ulteriori embedding da utilizzare come query per la ricerca nel database**

In [10]:
query_embedding = get_embedding("input_query.jpg")  # immagine da descrivere
D, I = index.search(query_embedding, k=1) #ricerca sul Db
retrieved_description = descriptions[I[0][0]] #recupero della descrizione dal risultato della ricerca
print("📥 Descrizione recuperata:", retrieved_description) #stampa del risultato trovato

#la stessa operazione è stata effettuata su altre due immagini, nel secondo caso si tratta sempre dell'immagine di un gatto diversa che restituirà lo stesso risultato della prima.
#La terza immagine raffigura un cane e quindi restituirà un risultato differente, collegandosi appunto all'immagine di un altro cane memorizzata nel db.

query_embedding2 = get_embedding("gatto_ghibli2.jpg")  # immagine da descrivere
D2, I2 = index.search(query_embedding2, k=1)
retrieved_description2 = descriptions[I2[0][0]]
print("📥 Descrizione recuperata:", retrieved_description2)

query_embedding3 = get_embedding("shiba.jpg")  # immagine da descrivere
D3, I3 = index.search(query_embedding3, k=1)
retrieved_description3 = descriptions[I3[0][0]]
print("📥 Descrizione recuperata:", retrieved_description3)

📥 Descrizione recuperata: Un gatto vestito da Babbo Natale
📥 Descrizione recuperata: Un gatto vestito da Babbo Natale
📥 Descrizione recuperata: Un cane di razza Shiba in stile Ghibli
